In [ ]:
import cv2
import numpy as np
import os

# Çıktı klasörü
output_folder = "puzzle_pieces"
os.makedirs(output_folder, exist_ok=True)

# Resmi ve maskeyi yükle
image = cv2.imread("image.jpg", cv2.IMREAD_UNCHANGED)
mask = cv2.imread("mask.png", cv2.IMREAD_GRAYSCALE)

# Resmi maskeye göre boyutlandır
image_resized = cv2.resize(image, (mask.shape[1], mask.shape[0]))

# Threshold uygula
_, binary_mask = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY)

# Açma (Opening) işlemi ile bağlantıları kopar
kernel = np.ones((5, 5), np.uint8)
binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)

# Konturları bul
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Küçük parçaları filtrele
min_area = 5000  # Çok küçük parçaları yok etmek için
filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

print(f"Son Bulunan Parça Sayısı: {len(filtered_contours)}")

# 🔹 Puzzle parçalarının dikdörtgen çerçevelerini al
bounding_boxes = [cv2.boundingRect(cnt) for cnt in filtered_contours]

# 🟢 ADIM 1: Y eksenine göre sıralama (satır bazlı)
sorted_rows = sorted(zip(bounding_boxes, filtered_contours), key=lambda b: b[0][1])

# 🟢 ADIM 2: X eksenine göre her satırı kendi içinde sıralama
grid_size = 10  # 10x10 puzzle
final_sorted_contours = []

for i in range(0, len(sorted_rows), grid_size):  # Her 10 parçalık grubu al
    row_group = sorted_rows[i:i + grid_size]  # Satırın parçalarını al
    row_group_sorted = sorted(row_group, key=lambda b: b[0][0])  # X eksenine göre sırala
    final_sorted_contours.extend(row_group_sorted)

# 🟢 Puzzle parçalarını oluştur
for index, (bounding_rect, contour) in enumerate(final_sorted_contours):
    row = (index // grid_size) + 1  # 1'den başlasın
    col = (index % grid_size) + 1   # 1'den başlasın

    # Yeni bir boş maske oluştur
    piece_mask = np.zeros_like(mask)

    # Konturu maske üzerine çiz
    cv2.drawContours(piece_mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Resimden parçayı al
    piece = cv2.bitwise_and(image_resized, image_resized, mask=piece_mask)

    # Şeffaflık için alfa kanalı ekleyelim
    b, g, r = cv2.split(piece)
    alpha = piece_mask
    piece_rgba = cv2.merge([b, g, r, alpha])

    # 🔹 🟢 ADIM: GEREKSİZ ALFA KANALINI KESME (Trim işlemi)
    coords = cv2.findNonZero(alpha)  # Şeffaf olmayan pikselleri bul
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)  # Opak alanın sınırlarını al
        piece_rgba = piece_rgba[y:y+h, x:x+w]  # Puzzle parçasını kırp

    # 🔹 Çıktıyı yeni formatta kaydet (örn: puzzle_piece_1_1.png)
    output_path = os.path.join(output_folder, f"puzzle_piece_{row}_{col}.png")
    success = cv2.imwrite(output_path, piece_rgba)

    if success:
        print(f"{output_path} başarıyla kaydedildi! ✅")
    else:
        print(f"HATA: {output_path} kaydedilemedi! ❌")

print("Tüm jigsaw puzzle parçaları başarıyla oluşturuldu ve kaydedildi! 🧩🎉")
